# Build an On-Disk Database
Using AnnSQL, there are two types of databases you can build. The first is a simple in-memory database for smaller datasets. The second is an on-disk database which we demonstrate how to build in this notebook. Building an on-disk AnnSQL database will allow you to query, filter, and run basic statistics on a laptop for larger than memory datasets.



###  Install the AnnSQL package

```bash
pip install annsql
```

### Import Libraries 

In [1]:
from AnnSQL import AnnSQL
from MakeDb import MakeDb
import scanpy as sc

### Load the dataset
Here, we load the sample pbmc3k raw dataset provided by Scanpy. **Note**: For very large datasets, it is necessary to open a dataset using the AnnData backed mode. Backed mode is fully supported. If opening in backed mode, the database will build in chunks. Depending on the size of your dataset and your compute source, this process may take time.

In [2]:
adata = sc.datasets.pbmc3k()
adata.var_names_make_unique()
print(adata)

AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'


### Build the AnnSQL database

In [3]:
#this delete command is for testing purposes only. Remove this line in production 
if os.path.exists("db/pbmc3k.asql"):
	os.remove("db/pbmc3k.asql")

MakeDb(adata=adata, db_name="pbmc3k", db_path="db/")

Time to make var_names unique:  19.29051899909973
Time to create X table schema:  0.2035069465637207
Time to insert X data:  8.828634023666382


### Open the Database
Below we instantiate the AnnSQL class with the db parameter pointing to our newly created database. By default the database files contain the `.asql` extension.

In [4]:
adata_sql = AnnSQL(db="db/pbmc3k.asql")

### Query the Database

In [5]:
adata_sql.query("SELECT * FROM X LIMIT 5")

,cell_id,MIR1302_10,FAM138A,OR4F5,RP11_34P13_7,RP11_34P13_8,AL627309_1,RP11_34P13_14,RP11_34P13_9,AP006222_2,...,KIR3DL2_1,AL590523_1,CT476828_1,PNRC2_1,SRSF10_1,AC145205_1,BAGE5,CU459201_1,AC002321_2,AC002321_1
0,AAACATACAACCAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AAACATTGAGCTAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AAACATTGATCAGC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AAACCGTGCTTCCG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AAACCGTGTATGCG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Calculate total counts per gene

In [6]:
#total counts per gene 
adata_sql.query("SELECT SUM(COLUMNS(*)) FROM (SELECT * EXCLUDE (cell_id) FROM X)")

,MIR1302_10,FAM138A,OR4F5,RP11_34P13_7,RP11_34P13_8,AL627309_1,RP11_34P13_14,RP11_34P13_9,AP006222_2,RP4_669L17_10,...,KIR3DL2_1,AL590523_1,CT476828_1,PNRC2_1,SRSF10_1,AC145205_1,BAGE5,CU459201_1,AC002321_2,AC002321_1
0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,116.0,70.0,0.0,0.0,0.0,0.0,0.0


### Normalize to 10k reads per library and log transform
Below, we illustrate how to do basic normalization and log transformations using AnnSQL. It's worth nothing that for smaller datasets, there are no runtime benefits over using AnnSQL. For larger datasets that are >50k cells; this method becomes more computationally feasible on a resource limited computer. 

In [7]:
#get all gene names 
gene_names = adata_sql.query(f"Describe X")['column_name'][1:].values

#add a total counts column
adata_sql.query(f"ALTER TABLE X ADD COLUMN total_counts FLOAT DEFAULT 0;")
print("Total counts column added")

#iterates gene_names in chunks
chunk_size = 990 #Ddb limited to 1k
for i in range(0, len(gene_names), chunk_size):
	chunk = gene_names[i:i+chunk_size]
	chunk = " + ".join(chunk) + " + total_counts"
	adata_sql.update_query(f"UPDATE X SET total_counts = ({chunk});", suppress_message=True)
print("Total counts added")

#normalize to 10k and log2
chunk_size = 200  #reduces db memory usage
for i in range(0, len(gene_names), chunk_size):
	updates = []
	chunk = gene_names[i:i + chunk_size]
	for gene in chunk:
		updates.append(f"{gene} = LOG2(({gene} / total_counts) * 1e4 + 1e-5)")
	update_query = f"UPDATE X SET {', '.join(updates)}"
	adata_sql.update_query(update_query, suppress_message=True)
print("Normalized to 10k and log2")

#show the first 5 rows
adata_sql.query("SELECT * FROM X LIMIT 5")

Total counts column added
Total counts added
Normalized to 10k and log2


,cell_id,MIR1302_10,FAM138A,OR4F5,RP11_34P13_7,RP11_34P13_8,AL627309_1,RP11_34P13_14,RP11_34P13_9,AP006222_2,...,AL590523_1,CT476828_1,PNRC2_1,SRSF10_1,AC145205_1,BAGE5,CU459201_1,AC002321_2,AC002321_1,total_counts
0,AAACATACAACCAC-1,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,...,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,2421.0
1,AAACATTGAGCTAC-1,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,...,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,4903.0
2,AAACATTGATCAGC-1,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,...,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,3149.0
3,AAACCGTGCTTCCG-1,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,...,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,2639.0
4,AAACCGTGTATGCG-1,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,...,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,-16.60964,981.0
